In [86]:
# imports
import pandas as pd
from numpy import nan
import matplotlib.pyplot as plt

import statsmodels
import statsmodels.api as sm

# Read data

In [87]:
# read data into a DataFrame
df = pd.read_csv("data/need_2021_anon_dataset_50k.csv")

print(df.shape)

(50000, 75)


In [88]:
df.head()

,PROP_TYPE,PROP_AGE_BAND,FLOOR_AREA_BAND,CONSERVATORY_FLAG,COUNCIL_TAX_BAND,IMD_BAND_ENG,IMD_BAND_WALES,REGION,LI_FLAG,LI_DATE,...,ElecValFlag2014,ElecValFlag2013,ElecValFlag2012,ElecValFlag2011,ElecValFlag2010,ElecValFlag2009,ElecValFlag2008,ElecValFlag2007,ElecValFlag2006,ElecValFlag2005
0,Mid terrace,2,3,NaN,B,2.0,NaN,E12000005,0,NaN,...,V,E,E,V,E,V,V,V,V,V
1,Flat,3,2,NaN,A,1.0,NaN,E12000008,0,NaN,...,V,V,V,V,V,V,V,V,V,V
2,Mid terrace,1,3,NaN,A,2.0,NaN,E12000001,0,NaN,...,V,V,V,V,V,V,V,V,V,V
3,Flat,4,1,0.0,A,3.0,NaN,E12000004,0,NaN,...,O,O,O,O,O,O,O,O,O,O
4,Semi detached,2,2,NaN,D,5.0,NaN,E12000008,0,NaN,...,V,V,V,V,V,V,V,V,V,V


In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 75 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PROP_TYPE          50000 non-null  object 
 1   PROP_AGE_BAND      50000 non-null  int64  
 2   FLOOR_AREA_BAND    50000 non-null  int64  
 3   CONSERVATORY_FLAG  7418 non-null   float64
 4   COUNCIL_TAX_BAND   50000 non-null  object 
 5   IMD_BAND_ENG       47410 non-null  float64
 6   IMD_BAND_WALES     2590 non-null   float64
 7   REGION             50000 non-null  object 
 8   LI_FLAG            50000 non-null  int64  
 9   LI_DATE            8568 non-null   float64
 10  CWI_FLAG           50000 non-null  int64  
 11  CWI_DATE           8412 non-null   float64
 12  PV_FLAG            50000 non-null  int64  
 13  PV_DATE            561 non-null    float64
 14  MAIN_HEAT_FUEL     50000 non-null  int64  
 15  Gcons2019          40329 non-null  float64
 16  Gcons2018          403

# Wrangle data
## select rows

In [90]:
# Select rows where the 'IMD_BAND_ENG' column is not NaN
df = df[df['IMD_BAND_ENG'].notna()]

print(df.shape)

(47410, 75)


## select columns

In [91]:
df = df[
    ['PROP_TYPE', 'PROP_AGE_BAND', 'FLOOR_AREA_BAND', 'COUNCIL_TAX_BAND',
     'IMD_BAND_ENG', 'REGION', 'LI_FLAG', 'CWI_FLAG', 'PV_FLAG', 'MAIN_HEAT_FUEL',
     'Gcons2019', 'Gcons2018', 'Gcons2017', 'Gcons2016', 'Gcons2015', 'Gcons2014', 'Gcons2013', 'Gcons2012', 'Gcons2011', 'Gcons2010', 'Gcons2009', 'Gcons2008', 'Gcons2007', 'Gcons2006', 'Gcons2005',
     'Econs2019', 'Econs2018', 'Econs2017', 'Econs2016', 'Econs2015', 'Econs2014', 'Econs2013', 'Econs2012', 'Econs2011', 'Econs2010', 'Econs2009', 'Econs2008', 'Econs2007', 'Econs2006', 'Econs2005']
]

print(df.shape)

(47410, 40)


## dealing with NaN

In [92]:
# Define the columns of interest
g_columns = [f'Gcons{i}' for i in range(2019, 2004, -1)]
e_clumns = [f'Econs{i}' for i in range(2019, 2004, -1)]

# Drop rows where both Gcons and Econs columns are NaN
df = df.dropna(subset=g_columns + e_columns, how='all')

print(df.shape)
df.info()

(47335, 40)
<class 'pandas.core.frame.DataFrame'>
Index: 47335 entries, 0 to 49999
Data columns (total 40 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PROP_TYPE         47335 non-null  object 
 1   PROP_AGE_BAND     47335 non-null  int64  
 2   FLOOR_AREA_BAND   47335 non-null  int64  
 3   COUNCIL_TAX_BAND  47335 non-null  object 
 4   IMD_BAND_ENG      47335 non-null  float64
 5   REGION            47335 non-null  object 
 6   LI_FLAG           47335 non-null  int64  
 7   CWI_FLAG          47335 non-null  int64  
 8   PV_FLAG           47335 non-null  int64  
 9   MAIN_HEAT_FUEL    47335 non-null  int64  
 10  Gcons2019         38309 non-null  float64
 11  Gcons2018         38318 non-null  float64
 12  Gcons2017         38206 non-null  float64
 13  Gcons2016         37731 non-null  float64
 14  Gcons2015         37236 non-null  float64
 15  Gcons2014         36954 non-null  float64
 16  Gcons2013         36639 non-null 

In [93]:
# fill NaN with 0 if both Gcons or Econs columns are NaN
condition1 = df.iloc[:, 10:25].isna().all(axis=1)
df.loc[condition1, df.columns[10:25]] = 0

condition2 = df.iloc[:, 25:].isna().all(axis=1)
df.loc[condition2, df.columns[25:]] = 0

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47335 entries, 0 to 49999
Data columns (total 40 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PROP_TYPE         47335 non-null  object 
 1   PROP_AGE_BAND     47335 non-null  int64  
 2   FLOOR_AREA_BAND   47335 non-null  int64  
 3   COUNCIL_TAX_BAND  47335 non-null  object 
 4   IMD_BAND_ENG      47335 non-null  float64
 5   REGION            47335 non-null  object 
 6   LI_FLAG           47335 non-null  int64  
 7   CWI_FLAG          47335 non-null  int64  
 8   PV_FLAG           47335 non-null  int64  
 9   MAIN_HEAT_FUEL    47335 non-null  int64  
 10  Gcons2019         46100 non-null  float64
 11  Gcons2018         46109 non-null  float64
 12  Gcons2017         45997 non-null  float64
 13  Gcons2016         45522 non-null  float64
 14  Gcons2015         45027 non-null  float64
 15  Gcons2014         44745 non-null  float64
 16  Gcons2013         44430 non-null  float64
 17

In [95]:
# fill other NaN with the mean of the column
df.iloc[:, 10:] = df.iloc[:, 10:].apply(lambda col: col.fillna(col.mean()), axis=0)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47335 entries, 0 to 49999
Data columns (total 40 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PROP_TYPE         47335 non-null  object 
 1   PROP_AGE_BAND     47335 non-null  int64  
 2   FLOOR_AREA_BAND   47335 non-null  int64  
 3   COUNCIL_TAX_BAND  47335 non-null  object 
 4   IMD_BAND_ENG      47335 non-null  float64
 5   REGION            47335 non-null  object 
 6   LI_FLAG           47335 non-null  int64  
 7   CWI_FLAG          47335 non-null  int64  
 8   PV_FLAG           47335 non-null  int64  
 9   MAIN_HEAT_FUEL    47335 non-null  int64  
 10  Gcons2019         47335 non-null  float64
 11  Gcons2018         47335 non-null  float64
 12  Gcons2017         47335 non-null  float64
 13  Gcons2016         47335 non-null  float64
 14  Gcons2015         47335 non-null  float64
 15  Gcons2014         47335 non-null  float64
 16  Gcons2013         47335 non-null  float64
 17

## change data type